# 1. Extracción y Transformación de los Datos
## Introducción
Antes de realizar cualquier análisis estadístico, es necesario ver la calidad y condición de nuestro datos. Es decir, qué tan **sucio** viene nuestro conjunto de datos. Por  **sucio** nos referimos simplemente a que pueden haber datos inconsistentes, datos duplicados, errores tipográficos o valores faltantes en nuestro conjunto de datos. Por lo tanto, se hace necesario una **transformación** y **limpieza** de nuestros datos, antes de realizar y aplicar cualquier modelo estadístico.    
Para ello, importamos las librerias que nos ayudaran con el manejo y tratamiento de los datos las cuales son **pandas** y **numpy**. 

In [9]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 

## 1.1 Carga del Datset
Utilizamos la función de pandas *pd.read_csv()* para leer y cargar nuestro data set.  Además utilizamos el metodo *.info()* , para ver la naturaleza de nuestro dataframe, o si existen columnas o con datos faltantes. 

In [10]:

df = pd.read_csv("C:\\Users\\robe3358\\OneDrive\\Desktop\\call-center-analysis\\data\\Call Center Data.csv")
df.info()
df.shape
df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Index                       1251 non-null   int64 
 1   Incoming Calls              1251 non-null   int64 
 2   Answered Calls              1251 non-null   int64 
 3   Answer Rate                 1251 non-null   object
 4   Abandoned Calls             1251 non-null   int64 
 5   Answer Speed (AVG)          1251 non-null   object
 6   Talk Duration (AVG)         1251 non-null   object
 7   Waiting Time (AVG)          1251 non-null   object
 8   Service Level (20 Seconds)  1251 non-null   object
dtypes: int64(4), object(5)
memory usage: 88.1+ KB


,Index,Incoming Calls,Answered Calls,Answer Rate,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG),Service Level (20 Seconds)
0,1,217,204,94.01%,13,0:00:17,0:02:14,0:02:45,76.28%
1,2,200,182,91.00%,18,0:00:20,0:02:22,0:06:55,72.73%
2,3,216,198,91.67%,18,0:00:18,0:02:38,0:03:50,74.30%
3,4,155,145,93.55%,10,0:00:15,0:02:29,0:03:12,79.61%
4,5,37,37,100.00%,0,0:00:03,0:02:06,0:00:35,97.30%
...,...,...,...,...,...,...,...,...,...
1246,1247,191,184,96.34%,7,0:00:07,0:02:50,0:01:56,92.55%
1247,1248,212,209,98.58%,3,0:00:10,0:02:51,0:01:45,89.10%
1248,1249,210,203,96.67%,7,0:00:12,0:03:22,0:03:52,85.24%
1249,1250,167,159,95.21%,8,0:00:16,0:03:16,0:02:42,83.03%


```{admonition} observación
:class: note 
Note que no hay elementos faltantes en nuestro dataset, ya que todos los campos tienen el mismo número de registros

### 1.2 Eliminación de columnas irrelevantes, conversión de datos de algunas columnas. 
* Eliminamos las columnas index y todas las que no son relevantes para el análisis.
* Limpiar columnas de porcentaje.
* Convertimos tiempo de espera a segundos. 
*  Validar consistencia en las llamadas:  llamadas atendidas + llamadas abandonadas = llamadas entrantes. 
* Filtrar filas con tasa de abandono de más del 50%.

In [11]:


# 1. Limpiar columnas de porcentaje
df["Answer Rate"] = df["Answer Rate"].str.replace("%", "").astype(float)
df["Service Level (20 Seconds)"] = df["Service Level (20 Seconds)"].str.replace("%", "").astype(float)

# 2. Convertir tiempos a segundos
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

time_columns = ["Answer Speed (AVG)", "Talk Duration (AVG)", "Waiting Time (AVG)"]
for col in time_columns:
    df[col + "_seconds"] = df[col].apply(time_to_seconds)

# 3. Validar consistencia en llamadas
df["Validation"] = df["Answered Calls"] + df["Abandoned Calls"]
inconsistent_rows = df[df["Validation"] != df["Incoming Calls"]]

# 4. Eliminar columna Index 
df = df.drop(columns=["Index"], errors='ignore')

# 5. Filtrar valores atípicos (ejemplo: filas con tasa de abandono > 50%)
df = df[(df["Abandoned Calls"] / df["Incoming Calls"]) <= 0.5]

# Mostrar resumen de limpieza
print(f"Filas inconsistentes encontradas: {len(inconsistent_rows)}")
print(f"Datos después de limpieza: {df.shape[0]} filas")


Filas inconsistentes encontradas: 0
Datos después de limpieza: 1243 filas



Observemos que despues de ir transformando nuestros datos y elimininado algunos datos atípicos, el tamaño de nuestro dataset va disminuyendo. Antes teníamos 1251 registros de los cuales ahora solo tenemos 1243. 


### 1.3 Creamos un nuevo dataset con las columnas que nos interesa estudiar
* Las relaciones que nos interesa estudiar son : **LLamadas Entrantes** y **Tiempo de espera**. Por lo tanto, creamos un nuevo dataframe con estas dos columnas con el metodo *.copy().* 

In [12]:
new_df = df[["Incoming Calls", "Waiting Time (AVG)_seconds"]].copy() 
new_df

,Incoming Calls,Waiting Time (AVG)_seconds
0,217,165
1,200,415
2,216,230
3,155,192
4,37,35
...,...,...
1246,191,116
1247,212,105
1248,210,232
1249,167,162


### 1.4 Renombramos las columnas de nuestro nuevo dataframe 

In [13]:
new_df= new_df.rename(columns={"Incoming Calls" : "Llamadas entrantes", "Waiting Time (AVG)_seconds": "Tiempo de espera promedio (Segundos)"})
new_df 


,Llamadas entrantes,Tiempo de espera promedio (Segundos)
0,217,165
1,200,415
2,216,230
3,155,192
4,37,35
...,...,...
1246,191,116
1247,212,105
1248,210,232
1249,167,162


### 

### 1.5 Eliminación de filas duplicadas
Debemos verificar si nuestro nuevo dataframe tiene filas duplicadas. Python tiene dos métodos para verificar la existencia de duplicados, uno de ellos es el método *.any()* que nos arroja un valor booleano. También Lo podemos verificar  con el método *.duplicated()*.

In [14]:
# Verificamos si hay duplicados
hay_duplicados = new_df.duplicated()
print("¿Hay duplicados?", hay_duplicados.any())

# Mostramos las filas duplicadas
filas_duplicadas = new_df[new_df.duplicated()]
print("Filas duplicadas:")
print(filas_duplicadas)

¿Hay duplicados? True
Filas duplicadas:
      Llamadas entrantes  Tiempo de espera promedio (Segundos)
99                   226                                   165
228                   36                                    91
261                   21                                    58
462                  139                                   183
563                  158                                   220
565                  151                                   338
585                   25                                    55
642                  155                                   138
710                  166                                   165
720                  167                                   169
721                  169                                   104
744                  192                                   220
1075                 150                                   131
1138                  30                                    46
1227           

```{admonition} Observación
:class: note
Debido a que existen 15 registros repetidos, procedemos a eliminarlos con el método  .drop_duplicates().
```

In [15]:
new_df = new_df.drop_duplicates()
new_df.shape
new_df 

,Llamadas entrantes,Tiempo de espera promedio (Segundos)
0,217,165
1,200,415
2,216,230
3,155,192
4,37,35
...,...,...
1246,191,116
1247,212,105
1248,210,232
1249,167,162


```{admonition} Importante  
:class: warning  
Despues del proceso de limpieza, observe que ahora nuestro dataframe tiene solamente 1228 registros debido a la eliminación de duplicados. Finalmente, una vez limpio nuestro dataframe, procedemos a guardarlo, y en el siguiente capítulo, haremos un análisis explotario de datos para detectar patrones y relaciones entre nuestras dos columnas. 
Recordemos que el objetivo es estudiar si existe influencia significativa entre el **número de llamadas entrantes** y **el tiempo de espera.**
```

In [16]:
# Guardar DataFrame en un archivo CSV
new_df[["Llamadas entrantes", "Tiempo de espera promedio (Segundos)"]].to_csv("datos_call_center.csv", index=False)